In [ ]:
import pandas as pd
import geopandas as gpd
import netCDF4 as nc
from scipy.io import netcdf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import os
from math import pi
import datetime
import rioxarray
from shapely.geometry import mapping
from shapely.geometry import Point
import rasterio

## Explore data
#### Data are predicted. For each file I select the 24steps with the next date. There are 24 images with size 400*480.

In [ ]:
test1 = xr.open_dataset('C:\\Users\\User\\Documents\\Projects_2022\\Perifereia_Attikis_2021_2024\\Phase4\\Hazard\\meteo\\WRF-20220601.grb2', engine='cfgrib')
test2 = test1.isel(step= slice(12,36))
test2
#test2 = test1.sel(step='name of value') e.g. test2 = test1.sel(time='2022-05-27T12:00:00.000000000')

In [ ]:
test2.time.values

#### Select the temperature values.

In [ ]:
test2.t2m.values 
#or 
#test2.t2m.values[:,:,:] #[step, latitude, longitude]

#### Select the first step (step=0) for every pixel (lat, lon= :,:).

In [ ]:
test2.t2m.values[0,:,:]

#### Select the first step (step=0) for the pixels with lat 0 and all lon. It is the first row of the previous array.


In [ ]:
test.max_temp.values.shape

In [ ]:
test2.t2m.values[0,0,:]

#### Select the first step (step=0) for the pixels with all lat and lon 0. It is the first column of the pre-previous array or else the first value of each line.

In [ ]:
test2.t2m.values[0,:,0]

#### Select the first step (step=0) for one pixel (lat=0,lon=0).

In [ ]:
test2.t2m.values[0,0,0]
#test2.t2m.values[1,0,300]

#### Select for all the steps (24 images) for one pixel (lat=390,lon=479). There are 24 values in the array.

In [ ]:
test2.t2m.values[:,390,479]

#### Select the max from the previous array, meaning the max temperature in a day for one pixel.

In [ ]:
test2.t2m.values[:,390,479].max()

In [ ]:
test2.t2m.values[0,390,479] #Temperature in one pixel for T00:00:00

#### Calculate the max temperature for every step.

In [ ]:
test3['max_temp']=test2.t2m.max(dim='step')
test3

In [ ]:
test3.max_temp.values

#### Show the max temperature for every step for one pixel. There is no step in max_temp, only (lat, lon).

In [ ]:
test3.max_temp.values[390,479] 

In [ ]:
test3.max_temp.values[:,479]

#### Check data -> As a result the max temperature in a day for each pixel is extracted.

In [ ]:
test2.t2m.values[:,390,479].max()

In [ ]:
test3.max_temp.values[390,479]

In [ ]:
test1.close()

## Processing for all the files/dates. - Convert hourly to daily.

In [ ]:
mypath = '/home/sg/Projects/perifereia/p4/wind'

In [ ]:
#Check if a file/date is missing
from os.path import exists
for date in datalist:
    print(date)
    file_exists = exists(os.path.join(mypath,'WRF-'+date+'.grb2'))
    if file_exists:
        print (True)
    else:
        print ('The file for ' +date+ ' does not exist')

In [ ]:
#Dates we want
start = datetime.datetime.strptime("20220527", "%Y%m%d")
end = datetime.datetime.strptime("20220930", "%Y%m%d")
date_generated = pd.date_range(start, end)
datelist = date_generated.strftime("%Y%m%d")
datelist

In [ ]:
#Or else use that
datetime_series = pd.Series(pd.date_range("2022-05-27", periods=127, freq="D"))
datetime_series

In [ ]:
file_list = [i for i in os.listdir('/home/sg/Projects/perifereia/p4/raw_data') if i.endswith('2')]

In [ ]:
meteo = xr.open_dataset(os.path.join('/home/sg/Projects/perifereia/p4/raw_data',file_list[0]), engine = 'cfgrib')

In [ ]:
meteo

In [ ]:
os.chdir('/home/sg/Projects/perifereia/p4/raw_data')

# for lazaros -- add dew temp

In [ ]:
for file in file_list:
    #print(date)
    #list_of_files = [os.path.join(mypath,'WRF-'+date+'.grb2')]
    #print(list_of_files)
    #try:
    meteo = xr.open_dataset(file, engine = 'cfgrib')
    date  = file.split('-')[1].split('.')[0]
    meteo_date = meteo.isel(step= slice(12,36))
    print('Day selected')
    #datetime_series = pd.Series(pd.date_range("2022-05-27", periods=127, freq="D"))
    #for time in datetime_series:
        #meteo_date = meteo_date.assign_coords({'time':time}) #If there is no time dimension, create one
        #print('Time dimension created')
    meteo_date['max_temp'] = meteo_date.t2m.max(dim='step')
    meteo_date['min_temp'] = meteo_date.t2m.min(dim='step')
    meteo_date['mean_temp'] = meteo_date.t2m.mean(dim='step')
    meteo_date['tp_daily'] = meteo_date.tp.sum(dim='step')
    print('Values calculated')
    out = meteo_date[['max_temp','min_temp','mean_temp','tp_daily']]
    out = out.assign_coords({"time": date})
    out = out.expand_dims('time')
    out.to_netcdf('/home/sg/Projects/perifereia/p4/meteo_nc/'+date+"_temp_prec.nc")
    print('File '+date+' saved') #The name of the file saved refers actually to the next day, as they are predicted data.
    #except:
    #    print('error')

#### For the precipitation parameter we need the sum of total percipitation from the previous 7 days (the amount of water the ground holds prior to a fire incident)

In [ ]:
mypath = '/home/sg/Projects/perifereia/p4/meteo_nc/' #Gets the list of all files and directories in a specified directory. 

In [ ]:
#Select only the nc files -referring to temperature and precipitation- from a folder
list_of_files = [os.path.join(mypath,i) for i in os.listdir(mypath) if i.endswith('_temp_prec.nc')] 

In [ ]:
print(list_of_files)
len(list_of_files)

In [ ]:
meteo = xr.open_mfdataset(list_of_files,concat_dim='time', combine='nested') 
#concat_dim='time' --> makes sure that there is a common coordinate to connect files
#combine='nested' --> makes sure that the files open one after another and they are all available for processing

In [ ]:
#Create a rolling window of size 7 days along the time dimension and sum over it
rolling_sum = meteo.tp_daily.rolling(time=7, min_periods=1).sum()
#the `min_periods=1` --> ensures that if there are fewer than 7 days of data available for a given date, it will still calculate the rolling sum using the available data.

In [ ]:
#Assign the rolling sum to a new variable
meteo['rain_7days'] = rolling_sum

In [ ]:
meteo.sel(time=meteo.time.values[0]).expand_dims('time')

In [ ]:
#To save the weekly sum of precipitation for each date
for i in meteo.time.values:
    print(i)
    #meteo = meteo.drop_vars('i') #to delete a coordinate that is not needed
    day = meteo.sel(time=i).expand_dims('time')
    #out = day[['max_temp','min_temp','mean_temp','tp_daily','rain_7days']]
    #print(out)
    #time_name = pd.to_datetime(str(i)).strftime('%Y%m%d')
    #print(time_name)
    out.to_netcdf('/home/sg/Projects/perifereia/p4/meteo_final/'+time_name+"_temp_7Dprec.nc")
    print('File '+time_name+'with temperature and 7days rain saved')

In [ ]:
meteo.time.values

In [ ]:
meteo.rain_7days[0,:,:].values

In [ ]:
meteo.rain_7days.values

In [ ]:
meteo.rain_7days.values.shape

In [ ]:
meteo.sel(time='2022-06-25T12:00:00.000000000',method='nearest').values
#method='nearest' --> there were no initial data for some dates, so this argument brings the date nearest to the selected one

In [ ]:
meteo.time.values

## Check if data are correct

In [ ]:
initial = xr.open_dataset('C:\\Users\\User\\Documents\\Projects_2022\\Perifereia_Attikis_2021_2024\\Phase4\\Hazard\\meteo\\WRF-20220602.grb2', engine='cfgrib')
first = initial.isel(step= slice(12,36))
first

In [ ]:
first.t2m.values[:,390,479]

In [ ]:
first.t2m.values[:,390,479].max()

In [ ]:
first.close()

In [ ]:
check = xr.open_dataset('C:\\Users\\User\\Documents\\Projects_2022\\Perifereia_Attikis_2021_2024\\Phase4\\Hazard\\meteo\\20220602_temp_prec.nc', engine='netcdf4')
check

In [ ]:
check.max_temp.values[390,479]

In [ ]:
check.close()

In [ ]:
check2 = xr.open_dataset('C:\\Users\\User\\Documents\\Projects_2022\\Perifereia_Attikis_2021_2024\\Phase4\\Hazard\\meteo\\20220602_temp_7Dprec.nc', engine='netcdf4')
check2

In [ ]:
check2.time.values

In [ ]:
check2.rain_7days.values

In [ ]:
check2.rain_7days.shape

In [ ]:
check2.close()

## Features to shapefile

In [ ]:
savepath = 'wind_results/'

In [ ]:
grid = gpd.read_file('grid/grid.shp')

In [ ]:
len(grid)

In [ ]:
#Assign a unique id to each cell in the grid
grid['id'] = range(1,(len(grid)+1))

In [ ]:
#grid = grid.explode(ignore_index=True, index_parts=None)
grid

In [ ]:
grid.drop(columns=['OBJECTID','OBJECTID_1','PageName','PageNumber','ORIG_FID'], inplace=True)

In [ ]:
grid = grid.astype({'id':'int'}) #to remove any decimals

In [ ]:
grid

In [ ]:
grid = grid.to_crs('EPSG:4326')
grid

In [ ]:
#Check crs of the grid shapefile
print("CRS: {}".format(grid.crs))

In [ ]:
print(grid.geometry.centroid.x)
grid.geometry.centroid.y

In [ ]:
#Assign the xy from geometry in separate columns
grid['longitude'] = grid.geometry.centroid.x
grid['latitude'] = grid.geometry.centroid.y
grid

In [ ]:
#Print xy from geometry
for point in grid['geometry']:
    print(point.xy[0][0],point.xy[1][0])

In [ ]:
points = grid[['longitude','latitude']]
points

In [ ]:
#Plot one variable from the nc meteodata files in comparison with the grid shapefile
ax = grid.plot(alpha=0.2, color='black')
check2.max_temp.plot(ax=ax,zorder=-1)

In [ ]:
#Clip to extent of grid shapefile after changing crs to the nc file and plot it
check2 = check2.rio.write_crs('EPSG:4326')
clipped = check2.rio.clip(grid.geometry.apply(mapping), grid.crs)
f, ax = plt.subplots(figsize=(10, 4))
clipped.max_temp.plot(ax=ax)
ax.set(title="NC Layer Cropped to Geodataframe Extent")
ax.set_axis_off()
plt.show()

In [ ]:
#Plot the clipped (to the extent of the grid shapefile) nc file in comparison to the grid shapefile
ax = grid.plot(alpha=0.2, color='black')
clipped.max_temp.plot(ax=ax,zorder=-1)

In [ ]:
#Check the process of combining just one nc file with the grid
final = pd.DataFrame([])
for index,row in grid.iterrows():
    print("id:", row.id)
    x = row.longitude
    y = row.latitude
    print(x,y)
    point_maxtemp = clipped.max_temp.sel(longitude=x,latitude=y,method = "nearest").values
    point_maxtemp = float(point_maxtemp)
    point_mintemp = clipped.min_temp.sel(longitude=x,latitude=y,method = "nearest").values
    point_mintemp = float(point_mintemp)
    point_meantemp = clipped.mean_temp.sel(longitude=x,latitude=y,method = "nearest").values
    point_meantemp = float(point_meantemp)
    point_7Drain = clipped.rain_7days.sel(longitude=x,latitude=y,method = "nearest").values
    point_7Drain = float(point_7Drain)
    df = pd.DataFrame(columns=['id', 'x', 'y', 'max_temp', 'min_temp', 'mean_temp', 'rain_7days'], index = [0])
    df['id']=row.id
    df['x']=x
    df['y']=y
    df['max_temp'] = point_maxtemp
    df['min_temp'] = point_mintemp
    df['mean_temp'] = point_meantemp
    df['rain_7days'] = point_7Drain
    #print("id: {} max_temp: {} x: {} y: {}".format(row.id, point_variables, x, y))
    print(df)
    final = pd.concat([final, df],axis=0) #Append every df created for each row
print(final)
final.to_csv((os.path.join(savepath,'final.csv')), index=False)
print('The final csv was created')

In [ ]:
clipped.sel(longitude=23.758767051663387, latitude=38.2543759424724, method='nearest').values #lon,lat from the first row in grid shapefile

In [ ]:
check2.max_temp.shape

In [ ]:
clipped.max_temp.shape

In [ ]:
#Sample of dates to check the process
#datelist = ['20220527','20220528']

In [ ]:
for date in datelist:
    print(date)
    try:
        files = [os.path.join(mypath,date+'_temp_7Dprec.nc')]
        meteodata = xr.open_mfdataset(files)
        #print(meteodata.max_temp.shape)
        #for feat in meteodata.variables:
            #print(feat)
        meteodata = meteodata.rio.write_crs('EPSG:4326')
        clipped = meteodata.rio.clip(grid.geometry.apply(mapping), grid.crs)
        #print(clipped.max_temp.shape)
        #print(clipped)
        final = pd.DataFrame([])
        for index,row in grid.iterrows():
            #print("id:", row.id)
            x = row.longitude
            y = row.latitude
            #print(x,y)
            point_maxtemp = clipped.max_temp.sel(longitude=x,latitude=y,method = "nearest").values
            point_maxtemp = float(point_maxtemp)
            point_mintemp = clipped.min_temp.sel(longitude=x,latitude=y,method = "nearest").values
            point_mintemp = float(point_mintemp)
            point_meantemp = clipped.mean_temp.sel(longitude=x,latitude=y,method = "nearest").values
            point_meantemp = float(point_meantemp)
            point_7Drain = clipped.rain_7days.sel(longitude=x,latitude=y,method = "nearest").values
            point_7Drain = float(point_7Drain)
            df = pd.DataFrame(columns=['id', 'x', 'y', 'date', 'max_temp', 'min_temp', 'mean_temp', 'rain_7days'], index = [0])
            df['id']=row.id
            df['x']=x
            df['y']=y
            #Because data are predicted the date could be changed inside the final csv (date=int(date) and date=date+1)
            df['date']=date
            df['max_temp'] = point_maxtemp
            df['min_temp'] = point_mintemp
            df['mean_temp'] = point_meantemp
            df['rain_7days'] = point_7Drain
            #print("id: {} max_temp: {} x: {} y: {}".format(row.id, point_maxtemp, x, y))
            #print(df)
            final = pd.concat([final, df],axis=0) #Append every df created for each row
        #print(final)
        final.to_csv((os.path.join(savepath,date+'_final.csv')), index=False)
        print('The final csv for '+date+' was created')
    except:
        print('error')

In [ ]:
file

In [ ]:
for date in datelist:
    print(date)
#    try:
    if os.path.isfile(os.path.join(savepath,date+'_wind.csv')):
        print('result exists')
        continue
    file = 'wind/'+date+'.nc'
    if not os.path.isfile(file):
        print('file does not exist')
        continue
    meteodata = xr.open_dataset(file)
    #print(meteodata.max_temp.shape)
    #for feat in meteodata.variables:
        #print(feat)
    meteodata = meteodata.rio.write_crs('EPSG:4326')
    clipped = meteodata.rio.clip(grid.geometry.apply(mapping), grid.crs)
    #print(clipped.max_temp.shape)
    #print(clipped)
    final = pd.DataFrame([])
    for index,row in grid.iterrows():
        #print("id:", row.id)
        x = row.longitude
        y = row.latitude
        #print(x,y)
        point_dom_dir = clipped.dom_dir.sel(lon=x,lat=y,method = "nearest").values
        point_dom_dir = int(point_dom_dir)
        point_dom_vel = clipped.dom_vel.sel(lon=x,lat=y,method = "nearest").values
        point_dom_vel = float(point_dom_vel)
        point_res_max = clipped.res_max.sel(lon=x,lat=y,method = "nearest").values
        point_res_max = float(point_res_max)
        point_dir_max = clipped.dir_max.sel(lon=x,lat=y,method = "nearest").values
        point_dir_max = int(point_dir_max)
        df = pd.DataFrame(columns=['id', 'x', 'y', 'date', 'dom_dir', 'dom_vel', 'res_max', 'dir_max'], index = [0])
        df['id']=row.id
        df['x']=x
        df['y']=y
        #Because data are predicted the date could be changed inside the final csv (date=int(date) and date=date+1)
        df['date']=date
        df['dom_dir'] = point_dom_dir
        df['dom_vel'] = point_dom_vel
        df['res_max'] = point_res_max
        df['dir_max'] = point_dir_max
        #print("id: {} max_temp: {} x: {} y: {}".format(row.id, point_maxtemp, x, y))
        #print(df)
        final = pd.concat([final, df],axis=0) #Append every df created for each row
    #print(final)
    final.to_csv((os.path.join(savepath,date+'_wind.csv')), index=False)
    print('The final csv for '+date+' was created')
#    except:
#        print('error')

In [ ]:
#Change date inside the final csv if the data are predicted
# csv_files = glob.glob(os.path.join(savepath, "*_final.csv"))
# for f in csv_files:
#     df = pd.read_csv(f)
#     #print(df)
#     initialdate = df.date.values[0]
#     print(initialdate)
#     actualdate = initialdate+1
#     print(actualdate)
#     initialdate = int(initialdate)
#     actualdate = int(actualdate)
#     df['date'] = df['date'].replace({initialdate:actualdate})
#     print(df)
#     df.to_csv(f, index=False)

#### Extras

In [ ]:
#Convert from Kelvin to Celsius
from scipy.constants import convert_temperature
import numpy as np
temp_cels = convert_temperature((test.max_temp.values), 'Kelvin', 'Celsius')
temp_cels

In [ ]:
plt.contourf(plot)
plt.colorbar()
#285-312K equals to 12-38Celsius